In [1]:
# 모듈 import

import os
import numpy as np
import random
import sys
sys.path.append("../mmdetection/")
# 이건 본인 환경의 mmdetection 폴더 잡아주면 됨

from mmengine.hooks import Hook
from mmengine.config import Config
from mmengine.runner import Runner
from mmdet.registry import DATASETS
from mmdet.utils import register_all_modules


from torch.utils.data import SubsetRandomSampler

In [2]:
import wandb
wandb.login(key='2a631ea744b03506a1330798e0724d1d917a821f')

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mevest71 (mevest71-boostcamp). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /data/ephemeral/home/.netrc


True

In [3]:
# 모든 모듈 등록
register_all_modules()

In [4]:
# config file 경로
cfg = Config.fromfile('../mmdetection/configs/faster_rcnn/faster-rcnn_r50_fpn_1x_coco.py')

# 데이터셋 경로
root='./dataset/'

# work_dir 경로 (log랑 체크포인트 저장할 곳)
root_work_dir='./work_dirs/faster_rcnn_r50_fpn_1x_wandb_og'

# train max epochs 수
train_epochs = 10

# 체크포인트 최대 몇 개 저장해놓을 건지
keep_ckpts = 3

# 체크포인트 인터벌 (체크포인트 몇번째마다 저장할 건지)
ckpts_interval = 1


# --------------------------wandb--------------------------

# wandb 프로젝트 이름 (안 바꾸는 걸 추천함, 개인적으로 자기 것만 모아놓고 싶으면 바꿔도 됨)
wandb_project_name = 'Project2'


# wandb 실험 이름 (이건 자기가 원하는 걸로 변경해야 알아보기 편하겠지요?)
wandb_experiment_name = '원하는 이름'


# 몇 iter(False로 한 경우)마다 로그를 저장할 건지
# wandb에 로깅하는 것 뿐만 아니라 log에도 이 인터벌만큼만 찍히므로 참고해서 입력할 것
# 0은 못 받음, 0보다 큰 수로 줘야 함
# 숫자를 작게 줄수록 로깅을 자주 함(돌려보고 정하면 됨)
log_interval = 200


# 모델에 따라 아래 쉘의 num_classes 지정하는 부분이나 몇 부분 더 확인해볼 것 (roi_head 써야 하는지 bbox_head에 바로 줘야 하는지 같은 거)

In [5]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# dataset config 수정
cfg.dataset_type = 'CocoDataset'
cfg.data_root = root

# Train dataset config 수정
cfg.train_dataloader = dict(
    batch_size=4,
    num_workers=2,
    persistent_workers=True,
    sampler=dict(type='DefaultSampler', shuffle=True),
    batch_sampler=dict(type='AspectRatioBatchSampler'),
    dataset=dict(
        type=cfg.dataset_type,
        data_root=cfg.data_root,
        ann_file='train.json',
        data_prefix=dict(img=''),
        filter_cfg=dict(filter_empty_gt=True, min_size=32),
        pipeline=cfg.train_pipeline,
        metainfo=dict(classes=classes)
    )
)

# Validation dataset config 수정
cfg.val_dataloader = dict(
    batch_size=1,
    num_workers=2,
    persistent_workers=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type=cfg.dataset_type,
        data_root=cfg.data_root,
        ann_file='test.json',
        data_prefix=dict(img=''),
        test_mode=True,
        pipeline=cfg.test_pipeline,
        metainfo=dict(classes=classes)
    )
)

# Test dataset config 수정 (validation과 동일하게 설정)
cfg.test_dataloader = cfg.val_dataloader

# Train, val, test evaluator 설정
cfg.train_evaluator = dict(
    type='CocoMetric',
    ann_file=cfg.data_root + 'train.json',
    metric='bbox',
    format_only=False
)

cfg.val_evaluator = dict(
    type='CocoMetric',
    ann_file=cfg.data_root + 'test.json',
    metric='bbox',
    format_only=False
)

cfg.test_evaluator = cfg.val_evaluator

# 기타 설정
cfg.train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=train_epochs, val_interval=1)
cfg.val_cfg = dict(type='ValLoop')
cfg.test_cfg = dict(type='TestLoop')

cfg.env_cfg = dict(
    cudnn_benchmark=False,
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0),
    dist_cfg=dict(backend='nccl'),
)

cfg.work_dir = root_work_dir

# 모델에 따라 선택하거나 아예 바꿔야 할 수도 있음
cfg.model.roi_head.bbox_head.num_classes = 10
#cfg.model.bbox_head.num_classes = 10

cfg.optim_wrapper.optimizer.lr = 0.02
cfg.optim_wrapper.clip_grad = dict(max_norm=35, norm_type=2)

In [6]:
# Hook, wandb, log, 시각화 관련 코드

# 체크포인트 pth와 로그 저장
cfg.default_hooks = dict(
    timer=dict(type='IterTimerHook'),
    param_scheduler=dict(type='ParamSchedulerHook'),
    checkpoint=dict(type='CheckpointHook', interval=ckpts_interval, max_keep_ckpts=keep_ckpts),
    sampler_seed=dict(type='DistSamplerSeedHook'),
    visualization=dict(type='DetVisualizationHook'),
    logger=dict(type='LoggerHook', 
                log_metric_by_epoch=False,
                interval=log_interval,
                )
)

wandb_kwargs = dict(
    project=wandb_project_name,
    name=wandb_experiment_name,
)

visualizer = dict(
    type='DetLocalVisualizer',
    vis_backends=[
        dict(type='LocalVisBackend'),
        dict(type='WandbVisBackend',
             init_kwargs=wandb_kwargs)
    ],
    name='visualizer'
)

cfg.visualizer = visualizer

In [7]:
# Runner 생성 및 학습 시작
runner = Runner.from_cfg(cfg)

runner.train()

10/15 14:00:43 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 1639735043
    GPU 0: Tesla V100-SXM2-32GB
    CUDA_HOME: None
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
    PyTorch: 1.12.1+cu116
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.6
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,cod

10/15 14:00:58 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
10/15 14:00:58 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                

KeyboardInterrupt: 